In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [13]:
import os
from rnn_utils import *

import torch
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SubsetRandomSampler

from sklearn.model_selection import ParameterGrid, ParameterSampler

import numpy as np
import pandas as pd
import json

from tqdm.notebook import tqdm

from config import Settings; settings = Settings()

from ICDMappings import ICDMappings
icdmap = ICDMappings()

import wandb
import torchmetrics.functional as f

idx = pd.IndexSlice

# Reproducibility

In [3]:
# Reproducibility
seed = settings.random_seed

np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Create dataset

In [ ]:
SubsetRandomSampler()

In [4]:
grouping = 'ccs'
batch_size=64

In [5]:
ccs_universe = list(icdmap.icd9_3toccs.data.keys())
dataset_folder = '/home/debian/Simao/master-thesis/data/model_ready_dataset/icare2021_diag_A301'
dataset = ICareDataset(os.path.join(dataset_folder,'dataset.json'),
                            ccs_universe,
                            grouping
                          )

train_dataloader = DataLoader(dataset,
                            batch_size=batch_size,
                            collate_fn=ICareCOLLATE(),
                            sampler=SubsetRandomSampler(dataset.train_indices)
                           )
val_dataloader = DataLoader(dataset,
                            batch_size=batch_size,
                            collate_fn=ICareCOLLATE(),
                            sampler=SubsetRandomSampler(dataset.val_indices)
                           )
test_dataloader = DataLoader(dataset,
                            batch_size=batch_size,
                            collate_fn=ICareCOLLATE(),
                            sampler=SubsetRandomSampler(dataset.test_indices)
                           )

# Nº batches
len(train_dataloader)
len(val_dataloader)
len(test_dataloader)

processing each patient


  0%|          | 0/262811 [00:00<?, ?it/s]

2875

616

616

In [27]:
t_dataloader = DataLoader(dataset,
                            batch_size=2,
                            collate_fn=ICareCOLLATE(),
                            sampler=SubsetRandomSampler([1,2])
                         )

In [28]:
next(iter(t_dataloader))['pids']

['00007823E1ED7B1C8BEA6C5F10ED1294', '0000D2D87A64B20092AFC7A6E6258AEF']

In [63]:
size = 2
np.random.seed(234)
np.random.randint(0,len(dataset),size=(size,))

array([41284, 23225])

In [37]:
size = 2
np.random.seed(234)
overfit_indices = np.random.randint(0,len(dataset),size=(size,))
small_dataloader = DataLoader(dataset,batch_size=2,collate_fn=ICareCOLLATE(),sampler=SubsetRandomSampler(overfit_indices))

In [38]:
next(iter(small_dataloader))['pids']

['168794FF65882C8346F52242D6845599', '281ED3CF55E810B9F98C5E51549B4F22']

In [68]:
pids = list()
for batch in iter(train_dataloader):
    pids.extend(batch['pids'])

In [71]:
len(pids)

183967

In [72]:
len(dataset)

262811

time to iterate the train dataset

In [30]:
%%timeit -r 1 -n 1
for batch in iter(train_dataloader):
    pass

5.56 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


test new dataloader stuff

In [10]:
n_labels = input_size = next(iter(train_dataloader))['history_mhot'][0].shape[-1]
model = RNN(input_size=n_labels,
          hidden_size=50,
          num_layers=1,
          n_labels=n_labels,
          model='rnn')
optim = Adam(model.parameters(),lr=0.01)

overfit_dataloader = DataLoader(dataset,
                                batch_size=2,
                                collate_fn=ICareCOLLATE(),
                                sampler=RandomSampler([0,1]))
batch = next(iter(train_dataloader))

train_one_epoch(model,train_dataloader,optim)



Starting to train each batch


0it [00:00, ?it/s]

0.035003167638960095

In [13]:
compute_dataloader_loss(model,train_dataloader)

forward passing each batch to compute the loss


0it [00:00, ?it/s]

tensor(0.0313)

In [ ]:
from torchmetrics.functional import f1_score

In [14]:
train_one_epoch(model,train_dataloader,optim)

Starting to train each batch


0it [00:00, ?it/s]

0.030051858050019843

In [15]:
compute_dataloader_loss(model,train_dataloader)

forward passing each batch to compute the loss


0it [00:00, ?it/s]

tensor(0.0302)

In [29]:
compute_metrics(model,train_dataloader)

0it [00:00, ?it/s]

{'recall@30': 0.6115867443603018,
 'precision@30': 0.05215703369482704,
 'f1@30': 0.09248651459942693,
 'recall': 0.040990288729172035,
 'precision': 0.06251492678932846,
 'f1': 0.045647173448830194}

In [8]:
print('Distribution of target size (nº of diagnostics in target window)')
all_lengths = {}
for e in tqdm(dataset.raw_data):
    lengths = [len(i) for i in dataset.raw_data[e]['ccs']['targets']]
    for l in lengths:
        if l not in all_lengths:
            all_lengths[l] = 1
        else:
            all_lengths[l] +=1
            
(pd.Series(all_lengths).sort_index() / pd.Series(all_lengths).sum())[:15].round(2)

Distribution of target size (nº of diagnostics in target window)


  0%|          | 0/262811 [00:00<?, ?it/s]

0     0.23
1     0.18
2     0.13
3     0.10
4     0.07
5     0.06
6     0.04
7     0.03
8     0.03
9     0.02
10    0.02
11    0.01
12    0.01
13    0.01
14    0.01
dtype: float64

# Overfit a batch

In [8]:
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence
from torch import nn
import matplotlib.pyplot as plt
from torchmetrics import functional as f

In [12]:
torch.manual_seed(231)
overfit_batch = next(iter(train_dataloader))

n_labels = input_size = next(iter(train_dataloader))['target_sequences']['sequence'].shape[2]

Uparams = dict(hidden_size=[25,50,100,150,200],
               num_layers=[1,2],
               lr=[0.0001,0.0004,0.004,0.01],
               model=['rnn','lstm','gru'])
params = ParameterGrid(Uparams)

for param_set in tqdm(params):
    
    config = {
      "batch": 1,
      "epochs": 100,
      "batch_size": 64,
      "name":"overfit_batch",
      "manual_seed":231,
        "hidden_size":param_set['hidden_size'],
        "num_layers":param_set['num_layers'],
        "model":param_set['model'],
        "lr":param_set["lr"]
    }
    
    wandb.init(project="karpathy-tips-icare", entity="snovaisg",config=config)

    model = RNN(input_size=input_size,
                  hidden_size=param_set['hidden_size'],
                  num_layers=param_set['num_layers'],
                  n_labels=n_labels,
                  model=param_set['model'])
    criterion = torch.nn.BCEWithLogitsLoss(reduction='none')
    optimizer = torch.optim.Adam(model.parameters(), lr=param_set['lr'])

    overfit_batch = next(iter(train_dataloader))

    losses = list()

    metrics = None
    for epoch in tqdm(range(101)): # 100 epochs
        log = dict()

        # compute loss 
        batch_loss = train_one_batch(model,overfit_batch,criterion,optimizer)
        loss = batch_loss['total_loss'] / (batch_loss['total_sequences'] * n_labels)

        # compute metrics
        batch_metrics = compute_metricsV3_batch(model,overfit_batch)
    
        log.update(batch_metrics)
        log.update({'loss':loss})
        log.update({'epoch':epoch})
        
        wandb.log(log)

        del log

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
f1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
f1@30,▁▁▁▁▁▂▁▁▁▂▂▂▂▂▂▂▃▃▃▄▄▄▄▄▅▆▆▆▆▇▇▇▇▇█▇▇▇██
loss,████▇▇▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▁▁
precision,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
precision@30,▁▁▁▁▁▂▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▆▆▆▆▇▇▇▇▇█▇▇▇██
recall,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
recall@30,▁▁▂▂▂▂▁▁▁▁▂▂▂▂▂▂▂▃▃▄▄▄▄▅▅▆▆▆▇▇▇▇▇▇█▇▇▇██
epoch,100
f1,0.0
f1@30,0.01783


  0%|          | 0/101 [00:00<?, ?it/s]

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
f1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁██████
f1@30,████████▇▇▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▃▃▃▂▂▂▂▂▁▁
loss,█████▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▂▂▂▁▁
precision,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁██████
precision@30,███████████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▃▃▃▂▂▂▂▂▁▁
recall,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁██████
recall@30,████▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▅▅▅▄▄▄▄▄▄▃▃▃▂▂▂▂▂▂▁▁
epoch,100
f1,0.00079
f1@30,0.00782


...

  0%|          | 0/101 [00:00<?, ?it/s]

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
f1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
f1@30,▁▁▄▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████████
loss,█▅▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
precision,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
precision@30,▁▁▄▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████████
recall,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
recall@30,▁▁▄▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██████████
epoch,100
f1,0.0
f1@30,0.10695


  0%|          | 0/101 [00:00<?, ?it/s]

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
f1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▃▃▃▄▄▄▄▅▆▆▆▇▇▇▇█
f1@30,▁▁▅▆▆▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇███████████████
loss,█▆▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
precision,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▃▃▄▄▄▄▅▅▅▆▆▇▇▇▇▇██
precision@30,▁▁▅▆▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇███████████████
recall,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▃▃▃▃▄▄▄▅▆▆▆▆▇▇▇█
recall@30,▁▁▅▆▆▆▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇█████████████
epoch,100
f1,0.35106
f1@30,0.14873


  0%|          | 0/101 [00:00<?, ?it/s]

# Train

In [38]:
n_labels = input_size = next(iter(train_dataloader))['target_sequences']['sequence'].shape[2]

criterion = torch.nn.BCEWithLogitsLoss(reduction='none')

In [36]:
hyperparameters = {
    'hidden_size':[100,150],
    'num_layers':[1],
    'lr':[0.01,0.02],
    'model':['rnn','gru']
    
}
meta_parameters = {
    'epochs':1
}

params = ParameterGrid(hyperparameters)
print(f'params:',len(params))

#random_params = ParameterSampler(params.param_grid,n_iter=len(params)-1,random_state=231)
#next(iter(random_params))

params: 8


# Test

In [ ]:
from torch import nn
import torch
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence, pack_sequence
import torch.nn.functional as F

In [ ]:
import torchmetrics

In [ ]:
from torchmetrics.functional import recall,precision,f1_score

In [ ]:
import pandas
import numpy

In [ ]:
param_set = next(iter(params))
config = param_set
model = RNN(input_size=input_size,
              hidden_size=config['hidden_size'],
              num_layers=config['num_layers'],
              n_labels=n_labels,
              model=config['model'])

optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])
criterion = torch.nn.BCEWithLogitsLoss(reduction='none')

# Improved masked stuff

In [140]:
def gen_mask_padded_loss(lengths,loss_shape):
    """
    This method creates a mask to later perform loss.masked_fill_(mask,0)
    
    Note: this method is called at each batch so it has been optimized to some extent
    sacrificing some readibility. Hence it may be criptic to understand everything.
    
    Parameters
    ----------
    lengths: list, shape = (batch_size,)
        list with the actual length of each sequence in the batch
    
    loss_shape: tuple/list, shape=(batch_size, max_seq_length, n_labels)
        shape of the loss tensor on a given batch
    """
    idx = pd.IndexSlice
    # i.e. [ (pos_in_batch, seq_size), ...]
    # e.g. imagine batch of two sequences. first has size 2 and second size 6. we get [(0,2),(1,6)]
    seq_size_per_seq = list(zip(range(0,len(lengths)),lengths.numpy()))

    # i.e. [ (pos_in_batch,seq_index),(pos_in_batch,seq_index),...]
    # e.g. imagine batch of two seqs. first has size 2, second has size 1. produces: [[(0,0),(0,1)],[(1,0)]]
    real_seq_pos_per_seq = [list(zip([a[0]]*a[1],range(0,a[1]))) for a in seq_size_per_seq]
    # just flattens the previous list.
    # i.e. (taking the previous example) produces: [(0,0),(0,1),(1,0)]
    real_seq_pos_per_seq = [item for seq in real_seq_pos_per_seq for item in seq] 

    # create a mask that initially has everything as True
    res = (pd.DataFrame(np.ones(shape=(loss_shape[0]*loss_shape[1],loss_shape[2])))
           .assign(seq=np.array([[seq] * loss_shape[1] for seq in range(len(lengths))]).reshape((-1,1)),
                   index=list(range(0,loss_shape[1]))*loss_shape[0]
                  )
           .set_index(['seq','index']) # index is mean to help in the .loc after this cascade
           .astype(bool) # all values of dataframe are set to False now.
          )
    
    # set to False the values we don't want to change (aka: values that are not paddings)
    res.loc[idx[real_seq_pos_per_seq],:] = False

    # stack from (batch_size*max_seq_length,n_labels) to (batch_size,max_seq_length,n_labels)
    mask = torch.tensor(res.to_numpy().reshape(loss_shape))
    
    # now the mask has the same shape as the loss and ready to be applied on torch.masked_fill_
    return mask

In [142]:
from torch.nn.utils.rnn import pad_packed_sequence

In [105]:
fpr_1, tpr, ths = f.roc(outs.view(-1,283)[:,diag_1],target_sequences.view(-1,283)[:,diag_1])
fpr_2, tpr, ths = f.roc(outs.view(-1,283)[:,diag_2],target_sequences.view(-1,283)[:,diag_2])

In [106]:
target_sequences.view(-1,283)[:,diag_1].sum()
target_sequences.view(-1,283)[:,diag_2].sum()

tensor(24.)

tensor(25.)

In [107]:
outs.view(-1,283)[:,diag_1]
outs.view(-1,283)[:,diag_2]

tensor([-4.1155, -3.4331, -3.0590,  ..., -0.1427, -0.1427, -0.1427],
       grad_fn=<SelectBackward0>)

tensor([-3.9595, -3.6251, -3.6365,  ..., -0.2735, -0.2735, -0.2735],
       grad_fn=<SelectBackward0>)

# gen_masked_padded_loss vs ignoring_outs

## setup

In [265]:
from torch.nn.utils.rnn import pad_packed_sequence

In [229]:
n_labels = input_size = next(iter(train_dataloader))['target_sequences']['sequence'].shape[2]
model = RNN(input_size=n_labels,
          hidden_size=50,
          num_layers=1,
          n_labels=n_labels,
          model='rnn')

criterion = torch.nn.BCEWithLogitsLoss(reduction='none')
criterion2 = torch.nn.BCEWithLogitsLoss(reduction='mean')
batch = next(iter(train_dataloader))
history,target = batch['train_sequences']['sequence'], batch['target_sequences']['sequence']

_,lens = pad_packed_sequence(history,batch_first=True)
outs = model(history)

## gen_masked_...

In [230]:
%%timeit -r 100 -n 1
loss = criterion(outs,target)
mask = gen_mask4padded_loss(lens,n_labels)
loss.masked_fill_(mask,0)
loss = loss.sum() / (lens.sum() * n_labels)
loss.backward(retain_graph=True)
model.zero_grad()

17.8 ms ± 1.72 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


## ignoring the outs

In [231]:
%%timeit -r 100 -n 1
non_padded_positions = gen_non_padded_positions(lens)

new_outs = outs.view(-1,n_labels)[sequence_positions,:]
new_target = target.view(-1,n_labels)[sequence_positions,:]

loss = criterion2(new_outs,new_target)
loss.backward(retain_graph=True)
model.zero_grad()

14.1 ms ± 765 µs per loop (mean ± std. dev. of 100 runs, 1 loop each)


In [232]:
torch

<module 'torch' from '/home/debian/Simao/miniconda3/envs/thesis/lib/python3.10/site-packages/torch/__init__.py'>

In [237]:
seq_sizes = torch.randint(1,32,size=(batch_size,))
seq_sizes


tensor([31, 20, 30,  9, 16,  1, 11, 10, 31, 25, 16, 20, 14, 27, 21,  2, 27,  4,
         8, 25, 14,  3, 26,  6, 25, 28, 25, 12, 20,  2,  7, 21,  6, 30,  4, 18,
        28,  1,  9, 15, 26, 17,  3, 11, 19, 28, 10, 18, 31,  4,  1, 18, 24, 31,
        27, 21, 21, 23, 11, 30, 13, 11, 13, 27])

In [283]:
from torch.nn.utils.rnn import pad_sequence,pack_sequence, pack_padded_sequence

In [ ]:
pack_padded_sequence()

In [241]:
pad_sequence(sequences,batch_first=True)

tensor([[6, 4, 9,  ..., 0, 0, 0],
        [6, 4, 2,  ..., 6, 1, 0],
        [8, 2, 5,  ..., 6, 8, 3],
        ...,
        [7, 3, 7,  ..., 0, 0, 0],
        [2, 6, 2,  ..., 0, 0, 0],
        [1, 1, 6,  ..., 0, 0, 0]])

In [250]:
batch_size=64
n_classes = 10
random_seq_sizes = torch.randint(low=1,high=32,size=(batch_size,))
sequences = [torch.rand(size=(size,n_classes)) for size in random_seq_sizes]
sequences_padded = pad_sequence(sequences,batch_first=True)

In [269]:
a = pack_sequence(sequences,enforce_sorted=False)

In [277]:
a.data.shape

torch.Size([1102, 10])

In [273]:
a.batch_sizes

tensor([64, 63, 60, 59, 58, 56, 52, 51, 49, 47, 45, 43, 42, 41, 40, 39, 36, 34,
        32, 29, 26, 23, 22, 21, 18, 16, 13,  9,  6,  4,  4])

In [261]:
targets = [torch.randint(0,2,size=(size,n_classes)) for size in random_seq_sizes]

In [263]:
torch.vstack(targets).shape

torch.Size([1102, 10])

In [256]:
torch.randint(0,1+1,size=(6,))

tensor([1, 0, 0, 0, 1, 1])

In [264]:
def gen_non_padded_positions(lengths):
    """
    Given the lengths of sequences in a batch. returns all the 
    non-padded positions of all sequences in the batch, assuming 
    the batch has the first dimension stacked from 
    (batch_size,max_seq_len,n_labels) -> 
    (batch_size * max_seq_len, n_labels).
    
    Example
    -------
    
    # init
    model = ...
    criterion = ...
    
    # create batch of sequences of random sizes
    batch_size=64
    n_classes = 10
    random_seq_sizes = torch.randint(low=1,high=32,size=(batch_size,))
    # create inputs
    sequences = [torch.rand(size=(size,n_classes)) for size in random_seq_sizes]
    # create targets - in this case its multi-label target, but doesn't have to be.
    targets = torch.vstack([torch.randint(0,2,size=(size,n_classes) for size in random_seq_sizes)])
    
    sequences_packed = pack_sequence(sequences,batch_first=True)
    outs = model(sequences_packed)
    
    non_padded_positions = gen_non_padded_positions(random_seq_sizes)
    
    non_padded_outs = outs.view(-1,n_classes)[non_padded_positions,:]
    
    loss = criterion(non_padded_outs,targets)
    
    loss.backward()
    
    """
    
    max_seq_length = max(lengths)

    seq_size_per_seq = list(zip(range(0,len(lengths)),lengths))

    # i.e. [ (pos_in_batch,seq_index),(pos_in_batch,seq_index),...]
    # e.g. imagine batch of two seqs. first has size 2, second has size 1. produces: [[(0,0),(0,1)],[(1,0)]]
    real_seq_pos_per_seq = [list(zip([a[0]]*a[1],range(0,a[1]))) for a in seq_size_per_seq]
    
    # just flattens the previous list.
    # i.e. (taking the previous example) produces: [(0,0),(0,1),(1,0)]
    real_seq_pos_per_seq = [item for seq in real_seq_pos_per_seq for item in seq] 
    
    # position of sequence when all sequences are stacked
    non_padded_positions = [i[0] * max_seq_length + i[1] for i in real_seq_pos_per_seq]
    
    return non_padded_positions


In [ ]:
torch.nn.BCEWithLogitsLoss()

In [182]:
new_outs = outs.view(-1,n_labels)[sequence_positions,:]
new_target = target.view(-1,n_labels)[sequence_positions,:]

In [183]:
criterion2 = torch.nn.BCEWithLogitsLoss()
loss_outs = criterion2(new_outs,new_target)

In [184]:
loss_outs.

tensor(0.6943, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [185]:
loss_padded

tensor(0.6943, grad_fn=<DivBackward0>)

In [186]:
loss_outs.backward(retain_graph=True)
gradients_outs = [torch.clone(e.grad) for e in model.parameters()]
model.zero_grad()
loss_padded.backward()
gradient_padded = [torch.clone(e.grad) for e in model.parameters()]

In [188]:
all([torch.all(gradient_padded[i] == gradients_outs[i]) for i in range(len(gradient_padded))])

False

In [196]:
gradients_outs[1] == gradient_padded[1]

tensor([[ True, False,  True,  ...,  True,  True, False],
        [False, False,  True,  ..., False, False,  True],
        [False, False, False,  ..., False, False, False],
        ...,
        [False,  True,  True,  ..., False,  True, False],
        [False, False, False,  ..., False, False, False],
        [ True,  True, False,  ...,  True,  True,  True]])

In [197]:
gradient_padded[1]

tensor([[-2.1620e-05,  8.8866e-05,  1.5719e-04,  ..., -8.7919e-05,
         -1.6597e-04,  7.5765e-05],
        [ 4.0710e-05, -9.8936e-05, -1.6286e-04,  ...,  1.0883e-04,
          1.5834e-04, -8.4390e-05],
        [ 8.0387e-06, -2.1700e-05,  2.0796e-06,  ...,  4.6779e-06,
          1.7564e-05, -8.5339e-06],
        ...,
        [ 1.0925e-04, -2.8380e-04, -4.3840e-04,  ...,  3.1460e-04,
          4.4019e-04, -2.2296e-04],
        [-4.6614e-06, -2.0706e-05, -3.0623e-05,  ...,  1.8251e-05,
          3.3621e-05, -1.9575e-05],
        [ 4.9178e-05, -1.1246e-04, -1.7171e-04,  ...,  1.3282e-04,
          1.7701e-04, -8.1494e-05]])

In [193]:
gradients_outs[1]

tensor([[-2.1620e-05,  8.8866e-05,  1.5719e-04,  ..., -8.7919e-05,
         -1.6597e-04,  7.5765e-05],
        [ 4.0710e-05, -9.8936e-05, -1.6286e-04,  ...,  1.0883e-04,
          1.5834e-04, -8.4390e-05],
        [ 8.0387e-06, -2.1700e-05,  2.0796e-06,  ...,  4.6779e-06,
          1.7564e-05, -8.5339e-06],
        ...,
        [ 1.0925e-04, -2.8380e-04, -4.3840e-04,  ...,  3.1460e-04,
          4.4019e-04, -2.2296e-04],
        [-4.6614e-06, -2.0706e-05, -3.0623e-05,  ...,  1.8251e-05,
          3.3621e-05, -1.9575e-05],
        [ 4.9178e-05, -1.1246e-04, -1.7171e-04,  ...,  1.3282e-04,
          1.7701e-04, -8.1494e-05]])

-----

## updated one now

In [89]:
%%timeit -r 100 -n 1
gen_mask_padded_lossV2(lens.numpy(),loss.shape)

1.02 ms ± 368 µs per loop (mean ± std. dev. of 100 runs, 1 loop each)


In [94]:
def gen_mask_padded_lossV2(lengths,loss_shape):
    """
    This method creates a mask to later perform loss.masked_fill_(mask,0)
    
    Note: this method is called at each batch so it has been optimized to some extent
    sacrificing some readibility. Hence it may be criptic to understand everything.
    
    Parameters
    ----------
    lengths: list, shape = (batch_size,)
        list with the actual length of each sequence in the batch
    
    loss_shape: tuple/list, shape=(batch_size, max_seq_length, n_labels)
        shape of the loss tensor on a given batch
    """

    max_len = max(lengths)

    seq_size_per_seq = list(zip(range(0,len(lengths)),lengths))

    # i.e. [ (pos_in_batch,seq_index),(pos_in_batch,seq_index),...]
    # e.g. imagine batch of two seqs. first has size 2, second has size 1. produces: [[(0,0),(0,1)],[(1,0)]]
    real_seq_pos_per_seq = [list(zip([a[0]]*a[1],range(0,a[1]))) for a in seq_size_per_seq]
    # just flattens the previous list.
    # i.e. (taking the previous example) produces: [(0,0),(0,1),(1,0)]
    real_seq_pos_per_seq = [item for seq in real_seq_pos_per_seq for item in seq] 
    # position of sequence when stacking all sequences
    sequence_positions = [i[0] * max_len + i[1] for i in real_seq_pos_per_seq]

    # create mask
    mask = torch.ones((loss_shape[0] * loss_shape[1],loss_shape[2]), dtype=torch.bool)
    
    # fill positions with real sequences
    mask[sequence_positions,:] = False
    
    mask = mask.view(loss_shape)
    
    return mask

V2 is 38x faster. lets test if the gradients are the same

## Setup

In [278]:
n_labels = input_size = next(iter(train_dataloader))['target_sequences']['sequence'].shape[2]
model = RNN(input_size=n_labels,
          hidden_size=50,
          num_layers=1,
          n_labels=n_labels,
          model='rnn')

criterion = torch.nn.BCEWithLogitsLoss(reduction='none')
batch = next(iter(train_dataloader))
history,target = batch['train_sequences']['sequence'], batch['target_sequences']['sequence']

_,lens = pad_packed_sequence(history,batch_first=True)
outs = model(history)

loss = criterion(outs,target)

mask = gen_mask_padded_loss(lens,tuple(loss.shape))
maskV2 = gen_mask_padded_lossV2(lens,tuple(loss.shape))

PackedSequence(data=tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), batch_sizes=tensor([64, 62, 46, 35, 24, 20, 18, 15, 14, 12,  8,  7,  7,  6,  5,  5,  4,  3,
         2,  2,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1]), sorted_indices=tensor([26, 58, 41, 53, 16, 39, 52, 22, 44, 49,  3, 35,  6, 42, 19, 34, 38, 57,
        20, 43, 54,  7, 24,  1,  2,  0, 33, 61, 59, 36, 10, 12, 21, 55, 15, 63,
        46, 51, 56,  5, 30, 13, 27, 18, 37, 23, 62, 60,  8,  9, 11, 14, 17, 50,
        48, 47, 28, 45, 29, 31, 32,  4, 25, 40]), unsorted_indices=tensor([25, 23, 24, 10, 61, 39, 12, 21, 48, 49, 30, 50, 31, 41, 51, 34,  4, 52,
        43, 14, 18, 32,  7, 45, 22, 62,  0, 42, 56, 58, 40, 59, 60, 26, 15, 11,
        29, 44, 16,  5, 63,  2, 13, 19,  8, 57, 36, 55, 54,  9, 53, 3

## Original gen_mask_padded_loss (with pandas)

In [146]:
loss_original = loss.masked_fill(mask, 0)
loss_original = loss_original.sum() / np.prod(list(loss_original.shape))
loss_original.backward(retain_graph=True)

gradients_original = [torch.clone(e.grad) for e in model.parameters()]

## updated one now

In [147]:
model.zero_grad()

In [148]:
loss_v2 = loss.masked_fill(maskV2, 0)
loss_v2 = loss_v2.sum() / np.prod(list(loss_v2.shape))
loss_v2.backward()

gradients_v2 = [torch.clone(e.grad) for e in model.parameters()]

In [151]:
all([torch.all(gradients_original[i] == gradients_v2[i]) for i in range(len(gradients_original))])

True

awesome!

----

In [68]:
mask,a,b = gen_mask_padded_lossV2(lens,loss.shape)

tensor([False, False, False, False, False, False, False, False,  True])

In [61]:
a

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 6),
 (0, 7),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (2, 0),
 (2, 1),
 (3, 0),
 (3, 1),
 (3, 2),
 (3, 3),
 (3, 4),
 (3, 5),
 (3, 6),
 (3, 7),
 (3, 8),
 (4, 0),
 (4, 1),
 (5, 0),
 (5, 1),
 (5, 2),
 (5, 3),
 (5, 4),
 (5, 5),
 (5, 6),
 (5, 7),
 (5, 8),
 (6, 0),
 (6, 1),
 (6, 2),
 (6, 3),
 (6, 4),
 (6, 5),
 (6, 6),
 (6, 7),
 (6, 8),
 (6, 9),
 (6, 10),
 (6, 11),
 (6, 12),
 (6, 13),
 (6, 14),
 (6, 15),
 (6, 16),
 (6, 17),
 (6, 18),
 (6, 19),
 (6, 20),
 (6, 21),
 (6, 22),
 (6, 23),
 (6, 24),
 (7, 0),
 (7, 1),
 (8, 0),
 (8, 1),
 (8, 2),
 (8, 3),
 (8, 4),
 (8, 5),
 (9, 0),
 (9, 1),
 (9, 2),
 (9, 3),
 (10, 0),
 (10, 1),
 (10, 2),
 (10, 3),
 (10, 4),
 (10, 5),
 (10, 6),
 (10, 7),
 (10, 8),
 (11, 0),
 (11, 1),
 (11, 2),
 (11, 3),
 (12, 0),
 (12, 1),
 (12, 2),
 (12, 3),
 (12, 4),
 (12, 5),
 (12, 6),
 (12, 7),
 (13, 0),
 (13, 1),
 (14, 0),
 (14, 1),
 (14, 2),
 (14, 3),
 (14, 4),
 (14, 5),
 (14, 6),
 (14, 7),
 (15, 0),
 (15, 1),
 (15, 2),
 (16, 

In [57]:
mask

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [ True,  True,  True,  ...,  True,  True,  True],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])

In [58]:
lens

tensor([ 8,  4,  2,  9,  2,  9, 25,  2,  6,  4,  9,  4,  8,  2,  8,  3,  8, 10,
         2,  3,  6,  2,  4,  3,  3,  2,  4,  5,  3,  7, 25,  7, 16,  2,  3,  3,
        18,  2,  3,  4,  2,  3,  2, 18,  8,  2, 43,  5,  3, 18,  6,  9,  3,  4,
         5, 11,  5,  2, 23,  6,  2,  6,  4,  5])

In [42]:
sequence_positions = [i[0] * max_len + i[1] for i in real_seq_pos_per_seq]

In [36]:
loss.view(-1,loss.shape[-1]).shape

torch.Size([2688, 283])

In [31]:
real_seq_pos_per_seq

[(0, 0),
 (0, 1),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6),
 (1, 7),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (2, 5),
 (2, 6),
 (2, 7),
 (2, 8),
 (2, 9),
 (2, 10),
 (2, 11),
 (2, 12),
 (2, 13),
 (2, 14),
 (3, 0),
 (3, 1),
 (3, 2),
 (3, 3),
 (3, 4),
 (3, 5),
 (3, 6),
 (4, 0),
 (5, 0),
 (5, 1),
 (5, 2),
 (5, 3),
 (6, 0),
 (6, 1),
 (6, 2),
 (7, 0),
 (8, 0),
 (8, 1),
 (8, 2),
 (9, 0),
 (9, 1),
 (9, 2),
 (10, 0),
 (10, 1),
 (10, 2),
 (10, 3),
 (11, 0),
 (11, 1),
 (12, 0),
 (12, 1),
 (12, 2),
 (12, 3),
 (12, 4),
 (12, 5),
 (12, 6),
 (12, 7),
 (12, 8),
 (12, 9),
 (12, 10),
 (12, 11),
 (12, 12),
 (12, 13),
 (12, 14),
 (12, 15),
 (12, 16),
 (12, 17),
 (13, 0),
 (14, 0),
 (14, 1),
 (14, 2),
 (14, 3),
 (15, 0),
 (15, 1),
 (15, 2),
 (15, 3),
 (15, 4),
 (16, 0),
 (16, 1),
 (16, 2),
 (16, 3),
 (16, 4),
 (16, 5),
 (16, 6),
 (16, 7),
 (16, 8),
 (16, 9),
 (16, 10),
 (16, 11),
 (16, 12),
 (17, 0),
 (17, 1),
 (17, 2),
 (17, 3),
 (17, 4),
 (17, 5),
 (17, 6),
 (18, 0),
 (18, 1),
 

In [ ]:
def gen_mask_padded_lossV2(lengths,loss_shape):
    """
    This method creates a mask to later perform loss.masked_fill_(mask,0)
    
    Note: this method is called at each batch so it has been optimized to some extent
    sacrificing some readibility. Hence it may be criptic to understand everything.
    
    Parameters
    ----------
    lengths: list, shape = (batch_size,)
        list with the actual length of each sequence in the batch
    
    loss_shape: tuple/list, shape=(batch_size, max_seq_length, n_labels)
        shape of the loss tensor on a given batch
    """
    idx = pd.IndexSlice
    # i.e. [ (pos_in_batch, seq_size), ...]
    # e.g. imagine batch of two sequences. first has size 2 and second size 6. we get [(0,2),(1,6)]
    seq_size_per_seq = list(zip(range(0,len(lengths)),lengths.numpy()))

    # i.e. [ (pos_in_batch,seq_index),(pos_in_batch,seq_index),...]
    # e.g. imagine batch of two seqs. first has size 2, second has size 1. produces: [[(0,0),(0,1)],[(1,0)]]
    real_seq_pos_per_seq = [list(zip([a[0]]*a[1],range(0,a[1]))) for a in seq_size_per_seq]
    # just flattens the previous list.
    # i.e. (taking the previous example) produces: [(0,0),(0,1),(1,0)]
    real_seq_pos_per_seq = [item for seq in real_seq_pos_per_seq for item in seq] 

    # create a mask that initially has everything as True
    res = (pd.DataFrame(np.ones(shape=(loss_shape[0]*loss_shape[1],loss_shape[2])))
           .assign(seq=np.array([[seq] * loss_shape[1] for seq in range(len(lengths))]).reshape((-1,1)),
                   index=list(range(0,loss_shape[1]))*loss_shape[0]
                  )
           .set_index(['seq','index']) # index is mean to help in the .loc after this cascade
           .astype(bool) # all values of dataframe are set to False now.
          )
    
    # set to False the values we don't want to change (aka: values that are not paddings)
    res.loc[idx[real_seq_pos_per_seq],:] = False

    # stack from (batch_size*max_seq_length,n_labels) to (batch_size,max_seq_length,n_labels)
    mask = torch.tensor(res.to_numpy().reshape(loss_shape))
    
    # now the mask has the same shape as the loss and ready to be applied on torch.masked_fill_
    return mask

----

In [10]:
from torchmetrics import functional as f

def compute_metricsV3(model, dataloader):
    
    
    metrics = None
    for i,batch in tqdm(enumerate(iter(dataloader))):
        
        batch_metrics = compute_metricsV3_batch(model,batch)
        if metrics is None:
            metrics = dict()
            for key in batch_metrics:
                metrics[key] = list(batch_metrics[key])
        else:
            for key in batch_metrics:
                metrics[key].append(batch_metrics[key])
    
    for key in batch_metrics:
        metrics[key] = np.mean(metrics[key])
    
    return metrics

def compute_metricsV3_batch(model,batch):
    history_sequences, target_sequences = batch['train_sequences']['sequence'],batch['target_sequences']['sequence']
    outs = model(history_sequences,target_sequences)

    recall_30 = f.recall(outs.view(-1,283),target_sequences.int().view(-1,283),top_k=30,average='samples') * target_sequences.shape[0] * target_sequences.shape[1] / target_sequences.any(dim=-1).sum()
    precision_30 = f.precision(outs.view(-1,283),target_sequences.int().view(-1,283),top_k=30,average='samples') * target_sequences.shape[0] * target_sequences.shape[1] / target_sequences.any(dim=-1).sum()
    f1_30 = f.f1_score(outs.view(-1,283),target_sequences.int().view(-1,283),top_k=30,average='samples') * target_sequences.shape[0] * target_sequences.shape[1] / target_sequences.any(dim=-1).sum()

    recall = f.recall(outs.view(-1,283),target_sequences.int().view(-1,283),average='samples') * target_sequences.shape[0] * target_sequences.shape[1] / target_sequences.any(dim=-1).sum()
    precision = f.precision(outs.view(-1,283),target_sequences.int().view(-1,283),average='samples') * target_sequences.shape[0] * target_sequences.shape[1] / target_sequences.any(dim=-1).sum()
    f1 = f.f1_score(outs.view(-1,283),target_sequences.int().view(-1,283),average='samples') * target_sequences.shape[0] * target_sequences.shape[1] / target_sequences.any(dim=-1).sum()
    
    return {'recall@30':recall_30.item(),
            'precision@30':precision_30.item(),
            'f1@30':f1_30.item(),
            'recall':recall.item(),
            'precision':precision.item(),
            'f1':f1.item()
           }

    
    

____

In [ ]:
model_folder = 'tmp_models/'

In [39]:
#param_set = {
#          'hidden_size':100,
#          'num_layers':1,
##          'lr':0.01,
#          'model':'rnn'
#         }
for idx,param_set in tqdm(enumerate(params)):
    config = {**param_set, 
              **meta_parameters}
    
    wandb.init(
        project="icare", 
        config=config
    )
    
    model = RNN(input_size=input_size,
              hidden_size=config['hidden_size'],
              num_layers=config['num_layers'],
              n_labels=n_labels,
              model=config['model'])
    
    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])
    
    loss = compute_loss(model,train_dataloader)
    wandb.log({'epoch':0,'loss':loss})
    
    print('Training each epoch')
    for epoch in tqdm(range(1,config['epochs']+1)):
        
        loss = train_one_epochV2(model,train_dataloader,epoch,criterion,optimizer);
        wandb.log({'epoch':epoch,'loss':loss})
        
        
    
    train_metrics = compute_metricsV3(model,train_dataloader)
    train_metrics = {f'train_{k}':train_metrics[k] for k in train_metrics}
    
    val_metrics = compute_metricsV3(model,val_dataloader)
    val_metrics = {f'val_{k}':val_metrics[k] for k in val_metrics}    

    log = dict()

    log.update(train_metrics)
    log.update(val_metrics)

    wandb.log(log)
    
    model_name = str(param_set)

    hypp_save_path = os.path.join(model_folder, model_name+'_hyper_parameters.json')

    with open(hypp_save_path, "w") as file:
        json.dump(params, file)

    print('Hyperparameters saved!')
    
    weights_save_path = os.path.join(model_folder,model_name+"_weights")

    torch.save(model.state_dict(), 
               weights_save_path
              )
    print('Model saved!')

0it [00:00, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Starting to train each batch


0it [00:00, ?it/s]

NameError: name 'compute_metricsV3' is not defined